In [4]:
import torch
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Tesseract 버전 확인 및 설치 (Linux 환경 예시)
!tesseract --version
!pip install --upgrade pip
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract

import os
import random
import cv2
import pytesseract
from pytesseract import Output
import pandas as pd

# (필요한 경우) Windows 환경에서 Tesseract 실행 파일 경로 설정 예시:
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


Using device: cuda:3
/bin/sh: 1: tesseract: not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires torch<2.4,>=2.2, but you have torch 2.4.1 which is incompatible.
autogluon-multimodal 1.1.1 requires torchvision<0.19.0,>=0.16.0, but you have torchvision 0.19.1 which is incompatible.
Get:1 http://mirror.kakao.com/ubuntu bionic InRelease [242 kB]
Get:2 http://mirror.kakao.com/ubuntu bionic-updates InRelease [102 kB]         
Get:3 http://mirror.kakao.com/ubuntu bionic-backports InRelease [102 kB]       
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [158

In [5]:
def segment_image_with_tesseract(
    image_path: str,
    output_dir: str,
    mode: str = "line",   # "line" 또는 "word" 선택
    prefix: str = "seg",
    psm: int = 6,
    save_meta: bool = True
):
    """
    Tesseract OCR을 이용하여 이미지에서 라인(line) 또는 단어(word) 단위 세그먼트를 추출합니다.
    
    Parameters:
        image_path (str): 원본 문서 이미지 경로
        output_dir (str): 세그먼트 이미지를 저장할 폴더 경로
        mode (str): "line" 또는 "word" (세그먼트 단위 선택)
        prefix (str): 저장될 세그먼트 파일명 접두사
        psm (int): Tesseract의 page segmentation mode (기본값 6)
        save_meta (bool): 세그먼트 정보(메타)를 CSV 파일로 저장할지 여부
        
    Returns:
        segments_info (list): 각 세그먼트에 대한 (filename, x1, y1, x2, y2, text, conf) 정보가 담긴 리스트
    """
    # 출력 디렉토리 생성 (없으면 생성)
    os.makedirs(output_dir, exist_ok=True)
    
    # 이미지 로드 (OpenCV 사용)
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Could not read the image file: {image_path}")
    
    # 전처리: 그레이 스케일 변환 (필요에 따라 이진화나 블러 적용 가능)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Tesseract 설정: PSM 모드를 지정합니다.
    custom_config = f'--psm {psm}'
    
    # Tesseract OCR 실행: bounding box 및 텍스트 정보를 dict 형태로 얻습니다.
    data = pytesseract.image_to_data(gray, config=custom_config, output_type=Output.DICT)
    
    # 유효한 bounding box 필터링 (텍스트가 비어있지 않고, 신뢰도(conf)가 0보다 큰 경우)
    n_boxes = len(data['level'])
    valid_indices = []
    for i in range(n_boxes):
        text = data['text'][i]
        conf_value = data['conf'][i]
        # conf 값이 정수형이면 그대로 사용, 문자열이면 isdigit() 체크 후 변환
        if isinstance(conf_value, int):
            conf = conf_value
        else:
            if not conf_value.isdigit():
                continue
            conf = int(conf_value)
        if text.strip() != "" and conf > 0:
            valid_indices.append(i)
    
    # 세그먼트 정보를 저장할 리스트 초기화
    segments_info = []
    
    if mode == "word":
        # 단어 단위 세그먼트: 각 valid index별로 바로 bounding box로 자릅니다.
        for i in valid_indices:
            x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
            crop_img = image[y:y+h, x:x+w]
            
            segment_filename = f"{prefix}_word_{i}.png"
            segment_path = os.path.join(output_dir, segment_filename)
            cv2.imwrite(segment_path, crop_img)
            
            segments_info.append({
                "segment_file": segment_filename,
                "x1": x,
                "y1": y,
                "x2": x + w,
                "y2": y + h,
                "text": data['text'][i].strip(),
                "conf": data['conf'][i]
            })
    else:
        # 라인 단위 세그먼트: 같은 line_num을 가진 bounding box들을 그룹화합니다.
        from collections import defaultdict
        line_groups = defaultdict(list)
        for i in valid_indices:
            line_idx = data['line_num'][i]
            line_groups[line_idx].append(i)
        
        # 각 라인 그룹별로 전체 bounding box를 계산하여 crop
        for line_idx, idx_list in line_groups.items():
            x1 = min([data['left'][j] for j in idx_list])
            y1 = min([data['top'][j] for j in idx_list])
            x2 = max([data['left'][j] + data['width'][j] for j in idx_list])
            y2 = max([data['top'][j] + data['height'][j] for j in idx_list])
            
            crop_img = image[y1:y2, x1:x2]
            
            segment_filename = f"{prefix}_line_{line_idx}.png"
            segment_path = os.path.join(output_dir, segment_filename)
            cv2.imwrite(segment_path, crop_img)
            
            # 그룹 내의 모든 텍스트를 결합 (단어 사이에 공백 추가)
            line_texts = [data['text'][j].strip() for j in idx_list]
            line_text = " ".join(line_texts)
            
            # 그룹 내 여러 bounding box의 신뢰도(conf) 값의 평균 계산
            conf_vals = (
                [int(data['conf'][j]) for j in idx_list if isinstance(data['conf'][j], str) and data['conf'][j].isdigit()] +
                [data['conf'][j] for j in idx_list if isinstance(data['conf'][j], int)]
            )
            avg_conf = sum(conf_vals) / len(conf_vals) if conf_vals else -1
            
            segments_info.append({
                "segment_file": segment_filename,
                "x1": x1,
                "y1": y1,
                "x2": x2,
                "y2": y2,
                "text": line_text.strip(),
                "conf": avg_conf
            })
    
    # 세그먼트 메타 정보 CSV로 저장 (옵션)
    if save_meta and segments_info:
        df_segments = pd.DataFrame(segments_info)
        meta_csv_path = os.path.join(output_dir, f"{prefix}_{mode}_segments.csv")
        df_segments.to_csv(meta_csv_path, index=False)
        print(f"Segments metadata saved: {meta_csv_path}")
    
    return segments_info


In [6]:
if __name__ == "__main__":
    # 원본 이미지 폴더: PNG 파일들이 위치한 폴더 경로
    input_dir = "/workspace/Hand/original"
    
    # 세그먼트 결과를 저장할 출력 폴더 경로
    output_dir = "/workspace/Hand/segments_test"
    os.makedirs(output_dir, exist_ok=True)
    
    # 폴더 내 모든 PNG 파일 목록 생성
    all_files = [f for f in os.listdir(input_dir) if f.lower().endswith(".png")]
    
    # 전체 파일이 많을 경우, 무작위로 10개만 샘플링
    sample_size = 10
    if len(all_files) > sample_size:
        sampled_files = random.sample(all_files, sample_size)
    else:
        sampled_files = all_files  # 파일이 10개 이하라면 모두 사용
    
    # 선택된 각 이미지에 대해 세그먼트 추출 수행
    for filename in sampled_files:
        image_path = os.path.join(input_dir, filename)
        
        # 파일명에서 확장자를 제거한 부분을 prefix로 사용
        prefix = os.path.splitext(filename)[0]
        
        segments_info = segment_image_with_tesseract(
            image_path=image_path,
            output_dir=output_dir,
            mode="line",  # "line" 또는 "word" 선택
            prefix=prefix,
            psm=6,
            save_meta=True
        )
        
        print(f"[Processed] {filename} -> {len(segments_info)} segments extracted.")


Segments metadata saved: /workspace/Hand/segments_test/w0683_s01_pPHR_r01_line_segments.csv
[Processed] w0683_s01_pPHR_r01.png -> 3 segments extracted.
Segments metadata saved: /workspace/Hand/segments_test/w0420_s03_pWOZ_r03_line_segments.csv
[Processed] w0420_s03_pWOZ_r03.png -> 3 segments extracted.
Segments metadata saved: /workspace/Hand/segments_test/w0085_s02_pLND_r02_line_segments.csv
[Processed] w0085_s02_pLND_r02.png -> 8 segments extracted.
Segments metadata saved: /workspace/Hand/segments_test/w0090_s03_pWOZ_r03_line_segments.csv
[Processed] w0090_s03_pWOZ_r03.png -> 7 segments extracted.
Segments metadata saved: /workspace/Hand/segments_test/w0590_s02_pLND_r01_line_segments.csv
[Processed] w0590_s02_pLND_r01.png -> 11 segments extracted.
Segments metadata saved: /workspace/Hand/segments_test/w0088_s02_pWOZ_r01_line_segments.csv
[Processed] w0088_s02_pWOZ_r01.png -> 7 segments extracted.
Segments metadata saved: /workspace/Hand/segments_test/w0636_s02_pLND_r02_line_segments